In [1]:
import pandas as pd 

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import nltk

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from keras.layers import GlobalAveragePooling1D,Merge,Lambda,Input,GlobalMaxPooling1D
from catboost import CatBoostClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

from sklearn.pipeline import make_pipeline
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords

/root/ml/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('17-11_train.csv')
df_test = pd.read_csv('17-11_test.csv')

In [3]:
df.shape

(283086, 257)

In [23]:
df_pos = df[(df['Revisit'] == 1)]
print("Total positives : ",df_pos.shape[0])
df_neg = df[(df['Revisit'] == 0)]
print("Total Negatives : ",df_neg.shape[0])
print("Base percentage of positives : ",100*df_pos.shape[0]/df_neg.shape[0])

Total positives :  234576
Total Negatives :  48510
Base percentage of positives :  483.5621521335807


In [24]:
# df.columns[:100]

In [25]:
df_pos = df_pos.sample(n=48510)

In [26]:
df_new = pd.concat([df_pos,df_neg]).sample(frac=1)
print(df_new.shape)

(97020, 257)


In [27]:
df = df_new

In [4]:
df.drop(['Unnamed: 0.1.1.1'], axis=1, inplace=True)

In [5]:
df_test.drop(['Unnamed: 0.1.1.1'], axis=1, inplace=True)

In [6]:
df.drop(['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace=True)
df_test.drop(['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace=True)

In [7]:
x_train = df[[e for e in df.columns.tolist() if e not in ['Revisit', 'Диагноз',  'Диагноз_morph', 'Код_диагноза', 
                            'Общее состояние_morph', 'Общее состояние',
                            'аллергия_morph', 'аллергия',
                            'Анамнез заболевания_morph', 'Анамнез заболевания',
                            'Внешний осмотр', 'Внешний осмотр_morph']]]
y_train = df['Revisit']

In [8]:
RandomForestClassifier(n_estimators=300, max_depth=3)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
df_test = df_test[[e for e in df_test.columns.tolist()
               if e not in ['Диагноз',  'Диагноз_morph', 'Код_диагноза', 
                            'Общее состояние_morph', 'Общее состояние',
                            'аллергия_morph', 'аллергия',
                            'Анамнез заболевания_morph', 'Анамнез заболевания',
                            'Внешний осмотр', 'Внешний осмотр_morph']]]

In [34]:
Xx_train, Xx_test, yy_train, yy_test = train_test_split(x_train, y_train, test_size=0.2, random_state=13)
rf = RandomForestClassifier(n_estimators=500, class_weight={0:1, 1:4.8})
rf.fit(Xx_train, yy_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 4.8},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [ ]:
# LogisticRegression

In [35]:
metrics.roc_auc_score(rf.predict(Xx_test), yy_test)

0.7227233414932457

In [4]:
# x_train_glove = df_glove_clear[[e for e in df_glove_clear.columns if e not in ['Revisit', 'Unnamed: 0']]]
# y_glove_train = df_glove_clear['Revisit']

In [36]:
vclf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=500)),
#         ('rf1', RandomForestClassifier(n_estimators=300, class_weight={0:1, 1:4.8})),
        ('lgb', lgb.LGBMClassifier(
            max_depth=7,
            n_estimators=200,
            learning_rate=0.01,
            num_leaves=16,
            reg_lambda=0.01)
        ),
        
        ('cat3', CatBoostClassifier(n_estimators=1000,
                          learning_rate=0.01,
                          eval_metric='AUC', 
#                           class_weight=[1, 4.8],
                          depth=3))
    ], 
    voting='soft'
)

vclf.fit(x_train, y_train)
# metrics.roc_auc_score(vclf.predict(df), df_test)

0:	learn: 0.7486131	total: 49ms	remaining: 48.9s
1:	learn: 0.7502315	total: 97.6ms	remaining: 48.7s
2:	learn: 0.7531758	total: 134ms	remaining: 44.5s
3:	learn: 0.7551263	total: 170ms	remaining: 42.3s
4:	learn: 0.7551305	total: 204ms	remaining: 40.6s
5:	learn: 0.7559762	total: 239ms	remaining: 39.6s
6:	learn: 0.7559762	total: 277ms	remaining: 39.4s
7:	learn: 0.7564915	total: 312ms	remaining: 38.7s
8:	learn: 0.7577121	total: 345ms	remaining: 38s
9:	learn: 0.7577162	total: 379ms	remaining: 37.6s
10:	learn: 0.7577162	total: 414ms	remaining: 37.3s
11:	learn: 0.7577162	total: 449ms	remaining: 37s
12:	learn: 0.7579272	total: 483ms	remaining: 36.7s
13:	learn: 0.7579272	total: 520ms	remaining: 36.7s
14:	learn: 0.7579272	total: 554ms	remaining: 36.4s
15:	learn: 0.7579272	total: 588ms	remaining: 36.1s
16:	learn: 0.7579272	total: 623ms	remaining: 36s
17:	learn: 0.7579272	total: 657ms	remaining: 35.9s
18:	learn: 0.7579272	total: 702ms	remaining: 36.2s
19:	learn: 0.7579272	total: 750ms	remaining: 36

162:	learn: 0.7795510	total: 6.53s	remaining: 33.5s
163:	learn: 0.7795648	total: 6.57s	remaining: 33.5s
164:	learn: 0.7795924	total: 6.61s	remaining: 33.5s
165:	learn: 0.7796137	total: 6.66s	remaining: 33.4s
166:	learn: 0.7796242	total: 6.7s	remaining: 33.4s
167:	learn: 0.7797308	total: 6.74s	remaining: 33.4s
168:	learn: 0.7797911	total: 6.8s	remaining: 33.4s
169:	learn: 0.7798265	total: 6.84s	remaining: 33.4s
170:	learn: 0.7798857	total: 6.88s	remaining: 33.4s
171:	learn: 0.7799911	total: 6.93s	remaining: 33.4s
172:	learn: 0.7800461	total: 6.98s	remaining: 33.4s
173:	learn: 0.7801584	total: 7.03s	remaining: 33.4s
174:	learn: 0.7803007	total: 7.09s	remaining: 33.4s
175:	learn: 0.7803617	total: 7.14s	remaining: 33.5s
176:	learn: 0.7803801	total: 7.2s	remaining: 33.5s
177:	learn: 0.7804236	total: 7.26s	remaining: 33.5s
178:	learn: 0.7803811	total: 7.32s	remaining: 33.6s
179:	learn: 0.7804942	total: 7.38s	remaining: 33.6s
180:	learn: 0.7805750	total: 7.43s	remaining: 33.6s
181:	learn: 0.7

322:	learn: 0.7869899	total: 14.3s	remaining: 30s
323:	learn: 0.7870147	total: 14.4s	remaining: 29.9s
324:	learn: 0.7870321	total: 14.4s	remaining: 29.9s
325:	learn: 0.7870541	total: 14.4s	remaining: 29.9s
326:	learn: 0.7870750	total: 14.5s	remaining: 29.8s
327:	learn: 0.7871153	total: 14.5s	remaining: 29.8s
328:	learn: 0.7871309	total: 14.6s	remaining: 29.7s
329:	learn: 0.7871577	total: 14.6s	remaining: 29.7s
330:	learn: 0.7871611	total: 14.7s	remaining: 29.7s
331:	learn: 0.7871824	total: 14.7s	remaining: 29.6s
332:	learn: 0.7872559	total: 14.8s	remaining: 29.6s
333:	learn: 0.7873012	total: 14.8s	remaining: 29.6s
334:	learn: 0.7873242	total: 14.9s	remaining: 29.5s
335:	learn: 0.7873219	total: 14.9s	remaining: 29.5s
336:	learn: 0.7873365	total: 15s	remaining: 29.4s
337:	learn: 0.7873690	total: 15s	remaining: 29.4s
338:	learn: 0.7874001	total: 15.1s	remaining: 29.4s
339:	learn: 0.7874681	total: 15.1s	remaining: 29.3s
340:	learn: 0.7875391	total: 15.1s	remaining: 29.3s
341:	learn: 0.7875

483:	learn: 0.7919552	total: 22.1s	remaining: 23.6s
484:	learn: 0.7919814	total: 22.2s	remaining: 23.6s
485:	learn: 0.7920313	total: 22.2s	remaining: 23.5s
486:	learn: 0.7920499	total: 22.3s	remaining: 23.5s
487:	learn: 0.7920798	total: 22.3s	remaining: 23.4s
488:	learn: 0.7921228	total: 22.4s	remaining: 23.4s
489:	learn: 0.7921618	total: 22.4s	remaining: 23.4s
490:	learn: 0.7921700	total: 22.5s	remaining: 23.3s
491:	learn: 0.7921891	total: 22.5s	remaining: 23.3s
492:	learn: 0.7921990	total: 22.6s	remaining: 23.2s
493:	learn: 0.7922139	total: 22.6s	remaining: 23.2s
494:	learn: 0.7922323	total: 22.7s	remaining: 23.1s
495:	learn: 0.7922557	total: 22.7s	remaining: 23.1s
496:	learn: 0.7922703	total: 22.8s	remaining: 23.1s
497:	learn: 0.7922838	total: 22.8s	remaining: 23s
498:	learn: 0.7923374	total: 22.9s	remaining: 23s
499:	learn: 0.7923587	total: 22.9s	remaining: 22.9s
500:	learn: 0.7923709	total: 23s	remaining: 22.9s
501:	learn: 0.7923964	total: 23s	remaining: 22.8s
502:	learn: 0.792415

644:	learn: 0.7958115	total: 30s	remaining: 16.5s
645:	learn: 0.7958254	total: 30s	remaining: 16.5s
646:	learn: 0.7958379	total: 30.1s	remaining: 16.4s
647:	learn: 0.7958515	total: 30.1s	remaining: 16.4s
648:	learn: 0.7958714	total: 30.2s	remaining: 16.3s
649:	learn: 0.7958909	total: 30.2s	remaining: 16.3s
650:	learn: 0.7959229	total: 30.3s	remaining: 16.2s
651:	learn: 0.7959687	total: 30.3s	remaining: 16.2s
652:	learn: 0.7959805	total: 30.4s	remaining: 16.2s
653:	learn: 0.7959990	total: 30.4s	remaining: 16.1s
654:	learn: 0.7960129	total: 30.5s	remaining: 16.1s
655:	learn: 0.7960263	total: 30.6s	remaining: 16s
656:	learn: 0.7960367	total: 30.6s	remaining: 16s
657:	learn: 0.7960597	total: 30.7s	remaining: 15.9s
658:	learn: 0.7961026	total: 30.7s	remaining: 15.9s
659:	learn: 0.7961200	total: 30.8s	remaining: 15.8s
660:	learn: 0.7961351	total: 30.8s	remaining: 15.8s
661:	learn: 0.7961481	total: 30.9s	remaining: 15.8s
662:	learn: 0.7961685	total: 30.9s	remaining: 15.7s
663:	learn: 0.796189

804:	learn: 0.7987736	total: 37.8s	remaining: 9.16s
805:	learn: 0.7987921	total: 37.9s	remaining: 9.11s
806:	learn: 0.7988070	total: 37.9s	remaining: 9.06s
807:	learn: 0.7988348	total: 37.9s	remaining: 9.02s
808:	learn: 0.7988565	total: 38s	remaining: 8.97s
809:	learn: 0.7988622	total: 38s	remaining: 8.92s
810:	learn: 0.7988745	total: 38.1s	remaining: 8.88s
811:	learn: 0.7988797	total: 38.1s	remaining: 8.83s
812:	learn: 0.7988877	total: 38.2s	remaining: 8.78s
813:	learn: 0.7989173	total: 38.2s	remaining: 8.74s
814:	learn: 0.7989516	total: 38.3s	remaining: 8.69s
815:	learn: 0.7989719	total: 38.3s	remaining: 8.64s
816:	learn: 0.7989795	total: 38.4s	remaining: 8.6s
817:	learn: 0.7990089	total: 38.5s	remaining: 8.56s
818:	learn: 0.7990218	total: 38.5s	remaining: 8.51s
819:	learn: 0.7990339	total: 38.6s	remaining: 8.47s
820:	learn: 0.7990495	total: 38.6s	remaining: 8.42s
821:	learn: 0.7990811	total: 38.7s	remaining: 8.38s
822:	learn: 0.7991052	total: 38.7s	remaining: 8.33s
823:	learn: 0.799

965:	learn: 0.8009810	total: 45.9s	remaining: 1.61s
966:	learn: 0.8009882	total: 45.9s	remaining: 1.57s
967:	learn: 0.8010058	total: 46s	remaining: 1.52s
968:	learn: 0.8010214	total: 46s	remaining: 1.47s
969:	learn: 0.8010299	total: 46s	remaining: 1.42s
970:	learn: 0.8010420	total: 46.1s	remaining: 1.38s
971:	learn: 0.8010723	total: 46.1s	remaining: 1.33s
972:	learn: 0.8010852	total: 46.2s	remaining: 1.28s
973:	learn: 0.8010914	total: 46.3s	remaining: 1.23s
974:	learn: 0.8011058	total: 46.3s	remaining: 1.19s
975:	learn: 0.8011190	total: 46.4s	remaining: 1.14s
976:	learn: 0.8011387	total: 46.4s	remaining: 1.09s
977:	learn: 0.8011505	total: 46.5s	remaining: 1.04s
978:	learn: 0.8011648	total: 46.5s	remaining: 999ms
979:	learn: 0.8011930	total: 46.6s	remaining: 951ms
980:	learn: 0.8012009	total: 46.7s	remaining: 904ms
981:	learn: 0.8012173	total: 46.7s	remaining: 856ms
982:	learn: 0.8012214	total: 46.7s	remaining: 808ms
983:	learn: 0.8012507	total: 46.8s	remaining: 761ms
984:	learn: 0.8012

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_we...n=200000, subsample_freq=1)), ('cat3', <catboost.core.CatBoostClassifier object at 0x7f594716e0f0>)],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [37]:
for_mean = pd.DataFrame(data={'proba': vclf.predict_proba(df_test)[:,1].ravel()}, index=df_test['ID'])

In [45]:
for_mean.to_csv('v9.csv')

In [65]:
x_train

,ID,Возраст,Пол,zeros,ones,Без динамики,Без дополнений,В течение последней недели беспокоят выделения из половых путей,Жалобы 2й день,Жалобы в течение 2-х недель. связывает с постоянной зрительной нагрузкой.,...,tfidf_char_nb_eap_Анамнез заболевания,tfidf_char_nb_eap_Внешний осмотр,count_words_nb_eap_Диагноз,count_words_nb_eap_Общее состояние,count_words_nb_eap_аллергия,count_words_nb_eap_Анамнез заболевания,count_words_nb_eap_Внешний осмотр,glove_cat_est500_lr05_d6,glove_lgb_est500_lr03_d7,glove_cat_est1000_lr03_d7
217247,217247,41,1,False,False,False,False,False,False,False,...,0.003020,1.944706e-01,1.566634e-01,1.718646e-01,0.171865,1.000000e+00,1.718646e-01,0.181731,0.815790,0.951967
23130,23130,29,2,False,False,False,False,False,False,False,...,0.000130,1.939608e-01,9.925377e-01,1.717674e-01,0.171767,1.795729e-14,1.717674e-01,0.322328,0.749919,0.967446
252432,252432,38,2,False,False,False,False,False,False,False,...,0.003225,1.927694e-01,9.245515e-01,1.711360e-01,0.171136,7.773361e-04,1.711360e-01,0.324605,0.695123,0.927064
271605,271605,34,2,False,False,False,False,False,False,False,...,0.014373,1.914799e-01,1.056200e-03,7.897144e-08,0.170766,1.707658e-01,1.707658e-01,0.049116,0.972755,0.980417
179440,179440,60,2,False,False,False,False,False,False,False,...,0.000083,1.927694e-01,3.983681e-02,1.711360e-01,0.171136,5.393399e-14,1.711360e-01,0.155733,0.838907,0.970573
209276,209276,32,1,False,False,False,False,False,False,False,...,0.000062,1.944706e-01,1.981894e-02,1.718646e-01,0.171865,1.030161e-15,1.718646e-01,0.177331,0.834008,0.948549
209796,209796,78,2,False,False,False,False,False,False,False,...,0.000105,1.939608e-01,4.396896e-01,1.717674e-01,0.171767,2.927218e-11,1.717674e-01,0.313524,0.657591,0.950869
123356,123356,32,2,False,False,False,False,False,False,False,...,0.002447,1.927694e-01,8.313662e-01,1.711360e-01,0.171136,3.153925e-03,1.711360e-01,0.271721,0.684728,0.937080
238387,238387,32,2,False,False,False,False,False,False,False,...,0.000065,1.927694e-01,5.323054e-04,1.711360e-01,0.171136,7.356875e-11,1.711360e-01,0.201353,0.791854,0.938308
233641,233641,47,2,False,False,False,False,False,False,False,...,0.001207,1.944706e-01,9.300258e-01,1.718646e-01,0.171865,2.334373e-02,1.718646e-01,0.316579,0.696914,0.927064


In [23]:
vclf2 = CatBoostClassifier(n_estimators=2000, learning_rate=0.01, eval_metric='AUC', depth=1)
vclf2.fit(x_train, y_train)

0:	learn: 0.6595700	total: 80.6ms	remaining: 2m 41s
1:	learn: 0.7025130	total: 164ms	remaining: 2m 44s
2:	learn: 0.7047194	total: 248ms	remaining: 2m 45s
3:	learn: 0.7096958	total: 333ms	remaining: 2m 45s
4:	learn: 0.7219827	total: 419ms	remaining: 2m 47s
5:	learn: 0.7219827	total: 503ms	remaining: 2m 47s
6:	learn: 0.7275004	total: 587ms	remaining: 2m 47s
7:	learn: 0.7275004	total: 673ms	remaining: 2m 47s
8:	learn: 0.7275004	total: 757ms	remaining: 2m 47s
9:	learn: 0.7275495	total: 842ms	remaining: 2m 47s
10:	learn: 0.7278310	total: 933ms	remaining: 2m 48s
11:	learn: 0.7293746	total: 1.06s	remaining: 2m 56s
12:	learn: 0.7294854	total: 1.19s	remaining: 3m 2s
13:	learn: 0.7294854	total: 1.3s	remaining: 3m 4s
14:	learn: 0.7310077	total: 1.42s	remaining: 3m 7s
15:	learn: 0.7310077	total: 1.53s	remaining: 3m 9s
16:	learn: 0.7310077	total: 1.64s	remaining: 3m 11s
17:	learn: 0.7310118	total: 1.75s	remaining: 3m 12s
18:	learn: 0.7310118	total: 1.85s	remaining: 3m 12s
19:	learn: 0.7319097	total

160:	learn: 0.7618182	total: 14.6s	remaining: 2m 47s
161:	learn: 0.7618182	total: 14.7s	remaining: 2m 47s
162:	learn: 0.7618182	total: 14.8s	remaining: 2m 47s
163:	learn: 0.7636111	total: 14.9s	remaining: 2m 46s
164:	learn: 0.7636111	total: 15s	remaining: 2m 46s
165:	learn: 0.7636111	total: 15.1s	remaining: 2m 46s
166:	learn: 0.7636223	total: 15.2s	remaining: 2m 46s
167:	learn: 0.7636224	total: 15.3s	remaining: 2m 46s
168:	learn: 0.7636224	total: 15.4s	remaining: 2m 46s
169:	learn: 0.7636224	total: 15.4s	remaining: 2m 46s
170:	learn: 0.7636224	total: 15.5s	remaining: 2m 46s
171:	learn: 0.7636224	total: 15.6s	remaining: 2m 46s
172:	learn: 0.7636224	total: 15.7s	remaining: 2m 46s
173:	learn: 0.7636224	total: 15.8s	remaining: 2m 46s
174:	learn: 0.7636197	total: 15.9s	remaining: 2m 45s
175:	learn: 0.7636197	total: 16s	remaining: 2m 45s
176:	learn: 0.7636197	total: 16.1s	remaining: 2m 45s
177:	learn: 0.7636197	total: 16.2s	remaining: 2m 45s
178:	learn: 0.7636197	total: 16.3s	remaining: 2m 4

316:	learn: 0.7668418	total: 29.8s	remaining: 2m 38s
317:	learn: 0.7670186	total: 29.9s	remaining: 2m 37s
318:	learn: 0.7670185	total: 29.9s	remaining: 2m 37s
319:	learn: 0.7670184	total: 30s	remaining: 2m 37s
320:	learn: 0.7670422	total: 30.1s	remaining: 2m 37s
321:	learn: 0.7670708	total: 30.3s	remaining: 2m 37s
322:	learn: 0.7670765	total: 30.4s	remaining: 2m 37s
323:	learn: 0.7671334	total: 30.5s	remaining: 2m 37s
324:	learn: 0.7671334	total: 30.6s	remaining: 2m 37s
325:	learn: 0.7671334	total: 30.7s	remaining: 2m 37s
326:	learn: 0.7671331	total: 30.8s	remaining: 2m 37s
327:	learn: 0.7671812	total: 30.9s	remaining: 2m 37s
328:	learn: 0.7671812	total: 31.1s	remaining: 2m 37s
329:	learn: 0.7671812	total: 31.2s	remaining: 2m 37s
330:	learn: 0.7672109	total: 31.3s	remaining: 2m 37s
331:	learn: 0.7672088	total: 31.4s	remaining: 2m 37s
332:	learn: 0.7672088	total: 31.5s	remaining: 2m 37s
333:	learn: 0.7672172	total: 31.6s	remaining: 2m 37s
334:	learn: 0.7672153	total: 31.7s	remaining: 2m

472:	learn: 0.7702217	total: 45.2s	remaining: 2m 25s
473:	learn: 0.7702186	total: 45.3s	remaining: 2m 25s
474:	learn: 0.7702328	total: 45.5s	remaining: 2m 25s
475:	learn: 0.7702328	total: 45.6s	remaining: 2m 25s
476:	learn: 0.7702329	total: 45.7s	remaining: 2m 25s
477:	learn: 0.7702540	total: 45.8s	remaining: 2m 25s
478:	learn: 0.7702493	total: 45.9s	remaining: 2m 25s
479:	learn: 0.7702478	total: 46.1s	remaining: 2m 25s
480:	learn: 0.7702900	total: 46.2s	remaining: 2m 25s
481:	learn: 0.7702939	total: 46.3s	remaining: 2m 25s
482:	learn: 0.7703052	total: 46.4s	remaining: 2m 25s
483:	learn: 0.7703255	total: 46.5s	remaining: 2m 25s
484:	learn: 0.7703213	total: 46.6s	remaining: 2m 25s
485:	learn: 0.7703271	total: 46.7s	remaining: 2m 25s
486:	learn: 0.7703271	total: 46.8s	remaining: 2m 25s
487:	learn: 0.7703266	total: 46.9s	remaining: 2m 25s
488:	learn: 0.7703263	total: 47s	remaining: 2m 25s
489:	learn: 0.7703253	total: 47.1s	remaining: 2m 25s
490:	learn: 0.7703453	total: 47.2s	remaining: 2m

628:	learn: 0.7718641	total: 1m 1s	remaining: 2m 13s
629:	learn: 0.7718635	total: 1m 1s	remaining: 2m 13s
630:	learn: 0.7718599	total: 1m 1s	remaining: 2m 13s
631:	learn: 0.7718684	total: 1m 1s	remaining: 2m 13s
632:	learn: 0.7718668	total: 1m 1s	remaining: 2m 13s
633:	learn: 0.7718791	total: 1m 1s	remaining: 2m 13s
634:	learn: 0.7718892	total: 1m 1s	remaining: 2m 13s
635:	learn: 0.7718886	total: 1m 2s	remaining: 2m 13s
636:	learn: 0.7719048	total: 1m 2s	remaining: 2m 12s
637:	learn: 0.7719186	total: 1m 2s	remaining: 2m 12s
638:	learn: 0.7719306	total: 1m 2s	remaining: 2m 12s
639:	learn: 0.7719382	total: 1m 2s	remaining: 2m 12s
640:	learn: 0.7719672	total: 1m 2s	remaining: 2m 12s
641:	learn: 0.7719711	total: 1m 2s	remaining: 2m 12s
642:	learn: 0.7719675	total: 1m 2s	remaining: 2m 12s
643:	learn: 0.7719660	total: 1m 2s	remaining: 2m 12s
644:	learn: 0.7719874	total: 1m 2s	remaining: 2m 12s
645:	learn: 0.7719845	total: 1m 3s	remaining: 2m 12s
646:	learn: 0.7719840	total: 1m 3s	remaining: 

...

1242:	learn: 0.7771293	total: 2m 7s	remaining: 1m 17s
1243:	learn: 0.7771563	total: 2m 7s	remaining: 1m 17s
1244:	learn: 0.7771457	total: 2m 7s	remaining: 1m 17s
1245:	learn: 0.7771504	total: 2m 7s	remaining: 1m 17s
1246:	learn: 0.7771440	total: 2m 7s	remaining: 1m 16s
1247:	learn: 0.7771444	total: 2m 7s	remaining: 1m 16s
1248:	learn: 0.7771652	total: 2m 7s	remaining: 1m 16s
1249:	learn: 0.7771666	total: 2m 7s	remaining: 1m 16s
1250:	learn: 0.7771907	total: 2m 7s	remaining: 1m 16s
1251:	learn: 0.7771947	total: 2m 7s	remaining: 1m 16s
1252:	learn: 0.7771880	total: 2m 8s	remaining: 1m 16s
1253:	learn: 0.7771797	total: 2m 8s	remaining: 1m 16s
1254:	learn: 0.7771866	total: 2m 8s	remaining: 1m 16s
1255:	learn: 0.7771920	total: 2m 8s	remaining: 1m 16s
1256:	learn: 0.7771910	total: 2m 8s	remaining: 1m 15s
1257:	learn: 0.7772161	total: 2m 8s	remaining: 1m 15s
1258:	learn: 0.7772075	total: 2m 8s	remaining: 1m 15s
1259:	learn: 0.7772315	total: 2m 8s	remaining: 1m 15s
1260:	learn: 0.7772261	total

1394:	learn: 0.7783076	total: 2m 23s	remaining: 1m 2s
1395:	learn: 0.7782959	total: 2m 23s	remaining: 1m 2s
1396:	learn: 0.7782986	total: 2m 23s	remaining: 1m 2s
1397:	learn: 0.7783268	total: 2m 23s	remaining: 1m 1s
1398:	learn: 0.7783175	total: 2m 24s	remaining: 1m 1s
1399:	learn: 0.7783228	total: 2m 24s	remaining: 1m 1s
1400:	learn: 0.7783448	total: 2m 24s	remaining: 1m 1s
1401:	learn: 0.7783372	total: 2m 24s	remaining: 1m 1s
1402:	learn: 0.7783331	total: 2m 24s	remaining: 1m 1s
1403:	learn: 0.7783365	total: 2m 24s	remaining: 1m 1s
1404:	learn: 0.7783268	total: 2m 24s	remaining: 1m 1s
1405:	learn: 0.7783304	total: 2m 24s	remaining: 1m 1s
1406:	learn: 0.7783261	total: 2m 24s	remaining: 1m 1s
1407:	learn: 0.7783428	total: 2m 25s	remaining: 1m
1408:	learn: 0.7783584	total: 2m 25s	remaining: 1m
1409:	learn: 0.7783862	total: 2m 25s	remaining: 1m
1410:	learn: 0.7783897	total: 2m 25s	remaining: 1m
1411:	learn: 0.7783895	total: 2m 25s	remaining: 1m
1412:	learn: 0.7783878	total: 2m 25s	remain

1548:	learn: 0.7793310	total: 2m 40s	remaining: 46.8s
1549:	learn: 0.7793232	total: 2m 40s	remaining: 46.7s
1550:	learn: 0.7793468	total: 2m 40s	remaining: 46.6s
1551:	learn: 0.7793595	total: 2m 40s	remaining: 46.5s
1552:	learn: 0.7793508	total: 2m 41s	remaining: 46.4s
1553:	learn: 0.7793561	total: 2m 41s	remaining: 46.3s
1554:	learn: 0.7793495	total: 2m 41s	remaining: 46.2s
1555:	learn: 0.7793429	total: 2m 41s	remaining: 46.1s
1556:	learn: 0.7793643	total: 2m 41s	remaining: 46s
1557:	learn: 0.7793889	total: 2m 41s	remaining: 45.9s
1558:	learn: 0.7793925	total: 2m 41s	remaining: 45.8s
1559:	learn: 0.7793850	total: 2m 41s	remaining: 45.7s
1560:	learn: 0.7794063	total: 2m 41s	remaining: 45.5s
1561:	learn: 0.7794277	total: 2m 42s	remaining: 45.4s
1562:	learn: 0.7794227	total: 2m 42s	remaining: 45.3s
1563:	learn: 0.7794136	total: 2m 42s	remaining: 45.2s
1564:	learn: 0.7794387	total: 2m 42s	remaining: 45.1s
1565:	learn: 0.7794623	total: 2m 42s	remaining: 45s
1566:	learn: 0.7794660	total: 2m

1702:	learn: 0.7801785	total: 2m 57s	remaining: 31s
1703:	learn: 0.7801962	total: 2m 58s	remaining: 30.9s
1704:	learn: 0.7802069	total: 2m 58s	remaining: 30.8s
1705:	learn: 0.7802062	total: 2m 58s	remaining: 30.7s
1706:	learn: 0.7802260	total: 2m 58s	remaining: 30.6s
1707:	learn: 0.7802940	total: 2m 58s	remaining: 30.5s
1708:	learn: 0.7802959	total: 2m 58s	remaining: 30.4s
1709:	learn: 0.7802887	total: 2m 58s	remaining: 30.3s
1710:	learn: 0.7803132	total: 2m 58s	remaining: 30.2s
1711:	learn: 0.7803118	total: 2m 58s	remaining: 30.1s
1712:	learn: 0.7803117	total: 2m 59s	remaining: 30s
1713:	learn: 0.7803323	total: 2m 59s	remaining: 29.9s
1714:	learn: 0.7803362	total: 2m 59s	remaining: 29.8s
1715:	learn: 0.7803403	total: 2m 59s	remaining: 29.7s
1716:	learn: 0.7803672	total: 2m 59s	remaining: 29.6s
1717:	learn: 0.7803692	total: 2m 59s	remaining: 29.5s
1718:	learn: 0.7803867	total: 2m 59s	remaining: 29.4s
1719:	learn: 0.7803806	total: 2m 59s	remaining: 29.3s
1720:	learn: 0.7803847	total: 2m

1858:	learn: 0.7812265	total: 3m 15s	remaining: 14.8s
1859:	learn: 0.7812429	total: 3m 15s	remaining: 14.7s
1860:	learn: 0.7812387	total: 3m 15s	remaining: 14.6s
1861:	learn: 0.7812303	total: 3m 15s	remaining: 14.5s
1862:	learn: 0.7812265	total: 3m 15s	remaining: 14.4s
1863:	learn: 0.7812256	total: 3m 15s	remaining: 14.3s
1864:	learn: 0.7812273	total: 3m 15s	remaining: 14.2s
1865:	learn: 0.7812203	total: 3m 16s	remaining: 14.1s
1866:	learn: 0.7812217	total: 3m 16s	remaining: 14s
1867:	learn: 0.7812252	total: 3m 16s	remaining: 13.9s
1868:	learn: 0.7812425	total: 3m 16s	remaining: 13.8s
1869:	learn: 0.7812575	total: 3m 16s	remaining: 13.7s
1870:	learn: 0.7813043	total: 3m 16s	remaining: 13.6s
1871:	learn: 0.7813240	total: 3m 16s	remaining: 13.4s
1872:	learn: 0.7813226	total: 3m 16s	remaining: 13.3s
1873:	learn: 0.7813156	total: 3m 16s	remaining: 13.2s
1874:	learn: 0.7813182	total: 3m 17s	remaining: 13.1s
1875:	learn: 0.7813180	total: 3m 17s	remaining: 13s
1876:	learn: 0.7813127	total: 3m

In [13]:
for_mean = pd.read_csv('v9.csv')

In [14]:
for_mean['proba2'] = vclf2.predict_proba(df_test)[:,1].ravel()

In [19]:
for_mean['v4'] = pd.read_csv('v4.csv')['proba']

In [20]:
for_mean['middle'] = (for_mean['v4'] + for_mean['proba2']) / 2

In [24]:
Submit_v12 = pd.DataFrame(data={'proba': vclf2.predict_proba(df_test)[:,1].ravel()}, index=df_test['ID'])

In [21]:
Submit_v11 = pd.DataFrame(data={'proba': for_mean['middle']}, index=df_test['ID'])

In [16]:
# Submit_v2

In [25]:
Submit_v12.to_csv('v12.csv')

In [155]:
df.columns[:30]

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'ID', 'Код_диагноза', 'Диагноз', 'Возраст', 'Пол', 'Общее состояние',
       'аллергия', 'Анамнез заболевания', 'Внешний осмотр', 'Revisit', 'zeros',
       'ones', 'Диагноз_morph', 'Общее состояние_morph', 'аллергия_morph',
       'Анамнез заболевания_morph', 'Внешний осмотр_morph', 'Без динамики',
       'Без дополнений',
       'В течение последней недели беспокоят выделения из половых путей',
       'Жалобы 2й день',
       'Жалобы в течение  2-х недель. связывает с постоянной зрительной нагрузкой.',
       'Жалобы несколько дней',
       'Жалобы несколько месяцев с усилением в последние 2 недели',
       'Жалобы около 2х месяцев', 'Консультация по проведенному обследованию'],
      dtype='object')

In [93]:
%%time
df['Анамнез заболевания_morph'].str.split(expand=True).stack().value_counts()

CPU times: user 8.85 s, sys: 3.1 s, total: 12 s
Wall time: 12 s


боль                    20596
течение                 16914
лечение                 15439
жалоба                  14603
беспокоить              12488
около                   11611
фон                     11495
обратиться              10700
отмечать                10526
год                      9962
обследование             9352
принимать                9256
последний                8700
день                     8165
появиться                7790
самостоятельно           7632
на                       7438
провести                 6958
дать                     6326
назад                    6326
неделя                   6115
консультация             6054
терапия                  6001
больной                  5808
результат                5760
положительный            5476
слово                    4989
ухудшение                4981
левый                    4950
приём                    4921
                        ...  
глаз+3,0).                  1
принимала:диклофенак        1
полидекса,

In [96]:
%%time
l_1 = df[df['Revisit'] == 1]['Анамнез заболевания_morph'].str.split(expand=True).stack().value_counts()

CPU times: user 4.34 s, sys: 1.73 s, total: 6.07 s
Wall time: 6.07 s


In [97]:
%%time
l_0 = df[df['Revisit'] == 0]['Анамнез заболевания_morph'].str.split(expand=True).stack().value_counts()

CPU times: user 4.94 s, sys: 1.62 s, total: 6.56 s
Wall time: 6.55 s


In [102]:
l_1['боль']

8795

In [103]:
for w, c in l_1.items():
    if w not in l_0:
        print(w, c)

уплотнить 10
корочки, 9
бокал 8
04.09.17г 8
субсерозный 7
585 7
11.03.16 7
таиланд 7
с4 7
кл, 6
19.01.2017 6
арутимол 6
(цифра 6
нед) 6
каудальный 6
синус, 6
мг/сутки. 6
983 6
количестве, 6
усталости, 6
яблока. 6
стодаль 5
21.02.17 5
10.00 5
семье. 5
сальпингит. 5
бад 5
контактом, 5
спкть 5
связываать 5
г): 5
подъм 5
28.01.16 5
неуточнить 5
постоянно) 5
солярия, 5
296 5
июне, 5
флексид 5
17.12.15 5
сустава? 5
запястно-пястный 5
созревание 5
(гипертонический 5
проблем. 5
ямке. 5
19.03.2017 5
эф., 5
гиперлипопротеинемия 5
синовиит. 5
25.12.16 5
698 5
нимесулида 5
чистотела, 5
с5/с6 5
постепеннный 5
медицины, 5
15.03.2017 5
лариста, 5
37,7), 5
амоксициклин, 5
канефрон- 5
-август 5
губах, 5
,по 5
пфс, 5
бакулева, 5
стойко 5
041 5
н\конечностей- 5
лимфома 5
крестце, 4
машины). 4
12.03.16 4
148 4
корреции 4
690 4
лазеркоагуляция 4
22.11 4
зостер 4
амбена 4
14.01.16г 4
печень, 4
мамой.самочувствие 4
07.01.16 4
нса 4
14.03.16. 4
слизь. 4
(норма) 4
обследована; 4
впс: 4
ксефокам. 4
(максимально

38,4с, 2
флотировать 2
изофру 2
аллергообследования. 2
383 2
уборки. 2
врачом: 2
листа, 2
25.10.17: 2
апоневроз 2
продолжительнее. 2
заниматься. 2
осложнений). 2
3-4раз 2
пыльца), 2
50-100 2
рос, 2
низкая. 2
голенью. 2
11.08.15 2
укб 2
артроскопия-удаление 2
жёстко 2
швейцария 2
элементов; 2
шваннома 2
-1.5д, 2
тонельный 2
гнойника, 2
120-140/60-08 2
небилонг 2
среду. 2
08.07.2017: 2
добавочный 2
самостоятель 2
ccадин 2
дриптан 2
перелома, 2
кровоостанавливающий 2
раз/год. 2
обследований 2
апф. 2
стенке? 2
(дважды) 2
э-9%, 2
теченией 2
васкулит? 2
эндометриоз) 2
адекватный. 2
31.07.15г. 2
уж 2
апизартрон 2
(утолщение 2
редкий. 2
стен 2
min 2
приоритет 2
2таблетка 2
14.07.2015. 2
слева?. 2
магнезию, 2
диафрагмы, 2
евстахеиты, 2
перешейка, 2
противирусный 2
02.08.17, 2
трим- 2
нейроциркуляторный 2
15-16 2
хвоста. 2
14.12.2017г. 2
14.05.17г 2
трифукация 2
14.07.16 2
(перелом). 2
ибс; 2
специальный 2
("вентолин" 2
принимаей 2
03.10.15 2
банки, 2
(эффективный 2
,дважды 2
18.09.15 2
11.09 2


3900, 2
03.01.2018, 2
4-5год 2
188954800171 2
нормализации, 2
смешанного? 2
5.96, 2
размеров). 2
целостности. 2
18.11.2016г 2
ж.п..сый 2
микроангиопатии, 2
кардиохирургический 2
29.03.2015 2
гематологом. 2
льняной 2
азизов 2
отохол 2
адрес 2
семя 2
тень 2
20.07.17г. 2
(дом. 2
учащено, 2
20.11.2016г, 2
нижегородский 2
3.11.16г. 2
нейт- 2
горн. 2
30.10.17, 2
плече-лучевой 2
руках)-ознакомить 2
". 2
нагноение 2
карсил, 2
имунал, 2
лет.лечиться 2
"медэлит-про" 2
+2,75 2
мазь") 2
иголка 2
05.03.18г-5 2
г\л. 2
уборщицей. 2
д; 2
другой, 2
никомед, 2
.вчера 2
("дезринит", 2
случаться 2
вит.д 2
аналогично 2
шейно- 2
21.05.17 2
д_з: 2
выявляли, 2
произвсть 2
01.12.2016г 2
стеклом, 2
врач-невролог 2
№289653743092 2
лопатками. 2
28.04.17. 2
01.2016г. 2
гиперурикемии. 2
урежается. 2
38,2гр, 2
ко-перинева. 2
(пролечен). 2
венчик 2
хондроксид. 2
анализов.ный 2
годов. 2
чёс 2
мелоксикам 2
электрофорез. 2
23.03.2017г 2
валвир, 2
прежних. 2
отекло. 2
внс. 2
волноорбазно, 2
140ь 2
абдоминопластика 2
(01)

31.01.16г 2
непроходимости. 2
тыс.). 2
8.04.17 2
п.зр.) 2
№52) 2
назад.раный 2
часах, 2
кс: 2
15.03.2016г, 2
рекомендаци 2
рл, 2
лечения.. 2
(обзорный 2
глаукому, 2
последнийя 2
крови(см. 2
ребенка), 2
наз-ада 2
отверстие, 2
жира; 2
нагрузками: 2
антациды- 2
перекрывать 2
464 2
11.00, 2
перемещениями. 2
коллаген, 2
с2-7, 2
перенагрузки, 2
теменно-височно-затылочный 2
предположительный 2
28.08.2015 2
ококло 2
было? 2
единоборства) 2
регтгенография 2
эпигатсрия. 2
тератозооспермия, 2
гипертонии. 2
2.2-2.7 2
лейк-2-6 2
привратника. 2
,дать 2
голени-стопы, 2
2000. 2
"какий-то 2
зрениия 2
даи) 2
насыщение 2
зарубежный 2
образований 2
гз 2
0,37 2
989 2
липокрем, 2
мазь; 2
дезлор, 2
иссха 2
=укол 2
расстройств. 2
аэропорт 2
кутузовском, 2
алфупрост, 2
назначени. 2
шоп- 2
обращался.обратиться 2
проходимы, 2
0,5х0,5 2
связок?) 2
перегородки, 2
4-5раз 2
-мелаксен 2
почсничный 2
02.05.16 2
антисептиками- 2
фасцита, 2
физкультуре. 2
(зарядка, 2
отказывалась. 2
сама. 2
зовиракс, 2
панкреатитом, 2
н

сдавала), 1
-4,0; 1
с4-5,. 1
исключены. 1
помогало. 1
нолодатак, 1
коагулограмма. 1
05.04.2016г.-5 1
1стлх3 1
нескоелько 1
подушка, 1
25.5(б14) 1
оки" 1
(лактационный 1
простатильный 1
ларинго-трахеит. 1
2313-16 1
04.07.2017) 1
антиоксандтный 1
аналогиченый 1
орви_- 1
пиона, 1
пр/ноги. 1
120. 1
2мг, 1
8.06.2015: 1
соэ-17 1
гиперваскуляризация 1
разростание 1
контроль.ть 1
заболеванияж, 1
влажный, 1
левомеколя. 1
20.08.17г. 1
куринный 1
альбумин- 1
нанесения, 1
(аг-2 1
диагнозом:диспепсия. 1
консультаацией. 1
xl, 1
22.08.16: 1
депиляция 1
2т-3 1
:эхо-признак 1
обняли. 1
ды), 1
врожд 1
брусья). 1
(th7-th12). 1
запах, 1
веловипеда 1
выскабливание...кариотип 1
(героин, 1
(05.07.2016 1
переломов: 1
кризы( 1
слева(1). 1
ш/о; 1
th7-th10 1
монопарез. 1
05.12.16-лапароскопия 1
асорил, 1
парааортальный 1
простанормом- 1
,урсосан. 1
дегенеративно-дистр.изменение 1
почек.считать 1
минздрав.российский 1
аденоидэктомия, 1
ранна 1
термо, 1
фстл-0- 1
ибупрофена, 1
(давно 1
коньковый 1
вонзникнуть 1
па

...

гипертоничека 1
бесспокит 1
позвоночника". 1
сустав(коленный 1
инфекция?). 1
улучшение:объем 1
анамнезе-эроивный 1
хондроин+ 1
положителоьный 1
(фервекс, 1
№72, 1
креветки, 1
8,2х14,1 1
повшение 1
(свето-звукочувствительность, 1
облать 1
(вытряхивание 1
полихимиотерапии. 1
выялить 1
3,5*2-3 1
(дн-4, 1
бассейне.аг 1
хлорфилипта, 1
лпу(документ 1
гиперемию. 1
(едкий 1
ухудшилось: 1
гептрал+ 1
аветисову. 1
парацетамолом, 1
142/83 1
пришли) 1
1-ого 1
неель 1
18.11.17: 1
двустонниеефральный 1
циннаризин. 1
химомицин, 1
1,2триместре, 1
эф, 1
провоцируют. 1
утра), 1
дислипидемия). 1
6,0d 1
люмбалгия,мышечно-тонический 1
проведен 1
отцом. 1
"уплотнение". 1
четвереньки 1
капс, 1
ретромбоз 1
70/30 1
5.85, 1
турции: 1
назад:ттг 1
т8-9. 1
заболеванию; 1
наконечником, 1
недели.когда 1
t7-t8-t9. 1
17.04.2016г, 1
36\3) 1
алоез- 1
подоюный 1
уреаплазмоз,? 1
деф. 1
15.02.16, 1
кинезиотейпирвание. 1
навпрлена 1
одестона, 1
а/б-кларитромицин, 1
нерва.прибыть 1
контроля,динамик 1
115/79 1
голвонай 1
пузыр

(шашалык, 1
утомляться 1
10.10.2017.- 1
сорт 1
повороте, 1
тп 1
губы) 1
(спкя), 1
фотография, 1
гангренозный 1
лдг 1
сстояние 1
дистинексом 1
гипогликемия? 1
районе. 1
приша 1
10/3 1
(клексана 1
регидром, 1
бх/кровь 1
(3,3-6,1) 1
спеуциалист 1
37х17х22 1
иссследование 1
гипертрофиовать 1
ципроцептом, 1
оксилометазолин 1
ад-130/80 1
диспное.врачом-аллергологом-иммунологом( 1
от2012 1
каешль 1
140-160/95 1
косточкой, 1
барьера) 1
(такролимус, 1
генерализация 1
спондилоартрита, 1
9,07.,после 1
тем. 1
мг./сут. 1
34*25 1
затруднено(переть 1
лишай". 1
метотрексат). 1
разошлись. 1
инсулинзависимый, 1
с6-с7) 1
резекци 1
сохраянуться 1
эг, 1
комбинил-дуо 1
какие, 1
го/э 1
п.гиса. 1
кожвендиспансер 1
"язык 1
.хронический 1
трахеостома 1
вентральной, 1
сильней. 1
позвонок, 1
бусит. 1
усилдиваться 1
грибки, 1
фаригосепт, 1
nivea, 1
беспричинно. 1
сипап-терапия, 1
вентрикуломегалия. 1
семечек, 1
ясенево. 1
(попоала 1
l2-l3,l3-l4,l4-l5,l5-s1. 1
азофен, 1
нолепрпел. 1
17.03.2017. 1
(криодеструкция) 1

кб№51 1
вернулаь. 1
шпоры». 1
онкогинеколог 1
спондилез).нарушение 1
инъекции). 1
марте) 1
кораксана. 1
дисменоея? 1
03.01.18- 1
травматолг 1
слоистость 1
хрноический 1
труда. 1
андроид). 1
фзе 1
мигренолом, 1
ингушетия 1
упражнениях. 1
04.02 1
31.12. 1
обострение?хронический 1
"распорка" 1
кифосколиотический 1
22.10.16 1
маслом). 1
кратким. 1
зажила. 1
брк. 1
2015-16 1
20.12.2017. 1
амброксол). 1
rpr 1
14.12.16г. 1
хр.запор 1
провыодить 1
оскорблять 1
описанный 1
/варикозный 1
№289653944531 1
амоксициллин)- 1
сохраняются.врачом-аллергологом-иммунолог 1
дюспаталин-с 1
плексопатия. 1
лимфопрессотерапия 1
перфоранты. 1
тянужие 1
вессел-дуэ-ф 1
гидроэнцефалия. 1
хирург) 1
30.03.17г 1
(джесс) 1
легче".ть 1
фоно, 1
десаметазон, 1
почек.клиновидный 1
3,70 1
138/94 1
эффекта.быть 1
/асептический 1
<50 1
нестабильный-склонность 1
таб./ный 1
вышежалобы. 1
херопрактику, 1
вдохе.дать 1
хр.вирусный 1
андипалом, 1
аквалор) 1
"заживают". 1
менингит? 1
l1/l2, 1
диклофенаком-время 1
полидексы, 1
экспи

огурцы, 1
целчок 1
(заноза) 1
валсартан+гидрохлортиазид 1
ds:"псориаз" 1
моментов. 1
абхазии. 1
авокадо), 1
иссхима 1
05.08.16. 1
.выписка 1
назад-халазиона 1
37,5градусов, 1
ключичный 1
бородавкий 1
в.конечность 1
мать). 1
насморк,повышение 1
одобрение 1
желанная; 1
страхов 1
прежней. 1
02.11.7 1
(28 1
поворотво 1
сходный 1
карипаин, 1
грудь\. 1
животе(после 1
3т 1
(дмж), 1
-3600 1
препарат), 1
диволь 1
ускорением. 1
мукоплант 1
682, 1
-сколько 1
геморроидэктомию. 1
лейк-3,6 1
рапид 1
2300, 1
стпени. 1
нр-положительный 1
2,3-2,8 1
резаня 1
пневмококк 1
возвращаются, 1
блефароконьюнктивит. 1
.но 1
-дэк 1
27.01.2017г, 1
обслдование 1
гигромы. 1
хиурргом. 1
артерий) 1
обед), 1
12.04.17. 1
5+500 1
часов- 1
07.11.2015: 1
тахикардия); 1
альмагеля, 1
нарушены; 1
голе. 1
погреб(1.5 1
охотой, 1
авф. 1
нлф, 1
68-80 1
10.02.2011 1
2700, 1
16.06.2015 1
16.101.15- 1
атипии.флора 1
th12-позвонка; 1
глазами: 1
полседневать 1
2014-похуделый 1
беременньсть 1
653. 1
4.6. 1
ксизал,кларитин, 1
назофаринг

18.07.16( 1
сохранялось, 1
ко-перинева, 1
экуссадтивный 1
2017), 1
года,, 1
недея 1
колебаниячий 1
назад.принимать 1
клебсиелла. 1
бассеина. 1
04.06.2017. 1
9.03.17 1
робщий 1
30.10.15, 1
тромбо-асс. 1
(формирование 1


In [117]:
not_in_l0 = []
for w, c in l_0.items():
    if w not in l_1 and c > 7:
        not_in_l0.append((w, c))

In [112]:
not_in_l1 = []
for w, c in l_1.items():
    if w not in l_0 and c > 5:
        not_in_l1.append((w, c))

In [119]:
prepare = not_in_l0 + not_in_l1 

In [126]:
for w, c in prepare:
    if isinstance(w, str):
        df[w] = df['Анамнез заболевания_morph'].apply(lambda x: w in x if isinstance(x, str) else False)

217247    False
23130     False
252432    False
271605    False
179440    False
209276    False
209796    False
123356    False
238387    False
233641    False
32589     False
61383     False
237940    False
176316    False
89813     False
56818     False
114421    False
5747      False
128097    False
77071     False
179368    False
114551    False
46480     False
137574    False
178326    False
7331      False
113845    False
75800     False
281779    False
84045     False
          ...  
149228    False
124284    False
168358    False
234330    False
87488     False
119380    False
166217    False
218701    False
205511    False
224782    False
243349    False
22207     False
191359    False
111294    False
220327    False
92715     False
26235     False
270125    False
124570    False
245683    False
250569    False
205524    False
219503    False
172853    False
163112    False
260423    False
115416    False
75304     False
163889    False
26469     False
Name: Анамнез заболевани

In [147]:
l_1 = df[df['Revisit'] == 1]['Анамнез заболевания_morph'].str.split(expand=True).stack().value_counts()
l_0 = df[df['Revisit'] == 0]['Анамнез заболевания_morph'].str.split(expand=True).stack().value_counts()

уплотнить 32
с4 24
11.03.16 24
37,7), 24
ципрамил 23
подвздошно-поясничный 21
нимесулида 21


In [152]:
not_in_l1 = []
for w, c in l_1.items():
    if w not in l_0 and c > 15:
        print(w, c)
        not_in_l1.append((w, c))
#         print(not_in_l1)
len(not_in_l1)

уплотнить 32
с4 24
11.03.16 24
37,7), 24
ципрамил 23
подвздошно-поясничный 21
нимесулида 21
афлубин, 20
корочки, 20
10.02.2017 20
585 19
стоп: 19
количестве, 19
аденомиоза. 19
21.09.2016 19
деликатно 19
остеотомия 18
возраста,пользоваться 18
("пульмикорт" 18
983 18
х/эктомия 18
23.12.16 18
уролесан, 18
нормально. 18
041 17
бад 17
28.01.16 17
созревание 17
922 17
безрезультатно. 16
926 16
контроля, 16
лимфома 16
мед/ть 16
15.09.15 16
базалиома 16
148 16
296 16
15.03.2017 16


39

In [157]:
def create_freq(df, df_test, f):
    l_1 = df[df['Revisit'] == 1][f].str.split(expand=True).stack().value_counts()
    l_0 = df[df['Revisit'] == 0][f].str.split(expand=True).stack().value_counts()
    
#     print(l_1)
#     print(l_0)
    
    not_in_l1 = []
    for w, c in l_1.items():
        if w not in l_0 and c > 10:
#             print(w, c)
            not_in_l1.append((w, c))
    print(len(not_in_l1))
    for w, c in tqdm(not_in_l1):
        if isinstance(w, str):
            df[w] = df[f].apply(lambda x: w in x if isinstance(x, str) else False)
            df_test[w] = df_test[f].apply(lambda x: w in x if isinstance(x, str) else False)
    return df, df_test

In [154]:
%%time
df, df_test = create_freq(df, df_test, 'Анамнез заболевания_morph')

  0%|          | 0/39 [00:00<?, ?it/s]

39


100%|██████████| 39/39 [00:06<00:00,  5.98it/s]


CPU times: user 28.3 s, sys: 5.25 s, total: 33.5 s
Wall time: 33.5 s


(        Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  Unnamed: 0.1.1.1      ID  \
 0                0             0               0                 0       0   
 1                1             1               1                 1       1   
 2                2             2               2                 2       2   
 3                3             3               3                 3       3   
 4                4             4               4                 4       4   
 5                5             5               5                 5       5   
 6                6             6               6                 6       6   
 7                7             7               7                 7       7   
 8                8             8               8                 8       8   
 9                9             9               9                 9       9   
 10              10            10              10                10      10   
 11              11            11              11   

In [160]:
df, df_test = create_freq(df, df_test, 'Диагноз_morph')

  6%|▋         | 1/16 [00:00<00:01,  8.31it/s]

16


100%|██████████| 16/16 [00:01<00:00,  8.20it/s]


In [158]:
%%time
df, df_test = create_freq(df, df_test, 'Общее состояние_morph')

  2%|▏         | 1/54 [00:00<00:05,  9.01it/s]

54


100%|██████████| 54/54 [00:06<00:00,  8.55it/s]

CPU times: user 13.2 s, sys: 948 ms, total: 14.2 s
Wall time: 14.1 s


In [159]:
%%time
df, df_test = create_freq(df, df_test, 'аллергия_morph')

  1%|▏         | 1/80 [00:00<00:08,  9.28it/s]

80


100%|██████████| 80/80 [00:08<00:00,  9.47it/s]

CPU times: user 12.1 s, sys: 492 ms, total: 12.6 s
Wall time: 12.5 s


In [161]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,...,аскариаз,дерматофития,хрящевой,[титце],послехирургический,стрептококковый,эктопический,"растениями,",рёберный,осложнять
0,0,0,0,0,0,J06.0,Острый ларингофарингит,29,2,NaN,...,False,False,False,False,False,False,False,False,False,False
1,1,1,1,1,1,N76.1,Подострый и хронический вагинит,45,2,NaN,...,False,False,False,False,False,False,False,False,False,False
2,2,2,2,2,2,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,NaN,...,False,False,False,False,False,False,False,False,False,False
3,3,3,3,3,3,N48.1,Баланопостит,43,2,NaN,...,False,False,False,False,False,False,False,False,False,False
4,4,4,4,4,4,Z00.0,Общий медицинский осмотр,29,1,NaN,...,False,False,False,False,False,False,False,False,False,False
5,5,5,5,5,5,Z01,"Другие специальные осмотры и обследования лиц,...",50,2,удовлетворительное.,...,False,False,False,False,False,False,False,False,False,False
6,6,6,6,6,6,M23.2,Поражение мениска в результате старого разрыва...,42,2,NaN,...,False,False,False,False,False,False,False,False,False,False
7,7,7,7,7,7,E05.0,Тиреотоксикоз с диффузным зобом,85,1,NaN,...,False,False,False,False,False,False,False,False,False,False
8,8,8,8,8,8,N77.1*,"Вагинит, вульвит и вульвовагинит при инфекцион...",28,2,Общее самочувствие удовлетворительное. Кожные ...,...,False,False,False,False,False,False,False,False,False,False
9,9,9,9,9,9,H60,Наружный отит,38,1,NaN,...,False,False,False,False,False,False,False,False,False,False


In [162]:
df.to_csv('18-11_train.csv')
df_test.to_csv('18-11_test.csv')